In [6]:
from stl import mesh
from mpl_toolkits import mplot3d
from matplotlib import pyplot
import pandas as pd
# Load the STL files and add the vectors to the plot
your_mesh = mesh.Mesh.from_file('data/meshlab.stl',remove_duplicate_polygons=1,calculate_normals=False,)
import numpy as np

In [7]:
normals=pd.DataFrame(your_mesh.normals)
toplayer_polygons=pd.DataFrame(your_mesh.points[normals[normals[2]>0.01].index])
selected_polygons=np.array(toplayer_polygons)

def area(x1, y1, x2, y2, x3, y3): 
  
    return abs((x1 * (y2 - y3) + x2 * (y3 - y1)  
                + x3 * (y1 - y2)) / 2.0) 
  
    
def isInside(x1, y1, x2, y2, x3, y3, x, y): 
    

    Area = 0.5 *(-y2*x3 + y1*(-x2 + x3) + x1*(y2 - y3) + x2*y3);
    s = 1/(2*Area)*(y1*x3 - x1*y3 + (y3 - y1)*x + (x1 - x3)*y);
    t = 1/(2*Area)*(x1*y2 - y1*x2 + (y1 - y2)*x + (x2 - x1)*y);
    if(s>=0 and t>=0 and (1-s-t)>=0):
        return True
    else:
        return False
    
    
def sign (x1, y1, x2, y2, x3, y3):
    return (x1- x3) * (y2 - y3) - (x2 - x3) * (y1 - y3);


def PointInTriangle (x1, y1, x2, y2, x3, y3, x, y): 


    d1 = sign(x,y, x1,y1,x2,y3);
    d2 = sign(x,y, x2,y2, x3,y3);
    d3 = sign(x,y, x3,y3, x1,y1);

    has_neg = (d1 < 0) or (d2 < 0) or (d3 < 0);
    has_pos = (d1 > 0) or (d2 > 0) or (d3 > 0);

    return not(has_neg and has_pos);
def pda(x,y,t):
    return sum(a*y+c*b+d*x<d*a+c*y+b*x for i in(0,1,2)for a,b,c,d in[t[i-1]+t[i]])%3<1
def threaded_function(cntr,x,y):
    for idx,_plgn in enumerate(selected_polygons):

        _x1,_y1,_z1,_x2,_y2,_z2,_x3,_y3,_z3=_plgn
        t=((_x1, _y1), (_x2, _y2), (_x3, _y3))
        if pda(x,y,t):
            #(PointInTriangle(_x1, _y1, _x2, _y2, _x3, _y3, x, y)):

            _pt=(x,y,np.mean([_z1,_z2,_z3]),your_mesh.normals[idx])
            point_grid[cntr]=(_pt)
            break#plt.plot(x,y,'b.')


In [9]:

x1=selected_polygons[:,0]
x2=selected_polygons[:,3]
x3=selected_polygons[:,6]
y1=selected_polygons[:,1]
y2=selected_polygons[:,4]
y3=selected_polygons[:,7]
x_max=np.max([np.max(x1),np.max(x2),np.max(x3)])
y_max=np.max([np.max(y1),np.max(y2),np.max(y3)])
x_min=np.max([np.min(x1),np.min(x2),np.min(x3)])
y_min=np.max([np.min(y1),np.min(y2),np.min(y3)])
dx=0.01
num_x=(x_max-x_min)//dx
point_grid=[]#[None]*(int(num_x**2))
cntr=0
threads=[]
from threading import Thread

num_x=20

for x in np.linspace(x_min, x_max,num_x):
    pnt_row=[]
    for y in np.linspace(y_min, y_max,num_x):
        
                #(PointInTriangle(_x1, _y1, _x2, _y2, _x3, _y3, x, y)):
        #thread = Thread(target = threaded_function, args = (cntr,x, y))
        #thread.start()
        #threads.append(thread)
        #cntr+=1;
        _pt=(None,None,None,None)
        
        for idx,_plgn in enumerate(selected_polygons):

            _x1,_y1,_z1,_x2,_y2,_z2,_x3,_y3,_z3=_plgn
            t=((_x1, _y1), (_x2, _y2), (_x3, _y3))
            if isInside(_x1, _y1, _x2, _y2, _x3, _y3, x, y):
                #(PointInTriangle(_x1, _y1, _x2, _y2, _x3, _y3, x, y)):

                _pt=(x,y,np.mean([_z1,_z2,_z3]),your_mesh.normals[idx])
                break#plt.plot(x,y,'b.')
        pnt_row.append(_pt)
    point_grid.append(pnt_row)

/home/dan/anaconda3/envs/cam/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in double_scalars
  from ipykernel import kernelapp as app
/home/dan/anaconda3/envs/cam/lib/python3.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in double_scalars
  app.launch_new_instance()


In [28]:
mesh=np.array(np.moveaxis(np.meshgrid(np.linspace(x_min, x_max,num_x),np.linspace(y_min, y_max,num_x)),0,-1))

In [59]:
import matplotlib.pyplot as plt
from sklearn.neighbors import KDTree
from scipy import spatial
%matplotlib inline
x,y=np.meshgrid(np.linspace(x_min, x_max,num_x),np.linspace(y_min, y_max,num_x))[0][:][:].ravel(),np.meshgrid(np.linspace(x_min, x_max,num_x),np.linspace(y_min, y_max,num_x))[1][:][:].ravel()
z=zip(x,y)
print(list(z).shape)

AttributeError: 'list' object has no attribute 'shape'

In [49]:
y.shape

(400,)

In [ ]:
tree = spatial.KDTree(zip(x.ravel(), y.ravel()))